## Lifetimes

- Lifetimes enforce that a piece of memory for a given reference is still valid.
- Lifetime for a given value (references only?) ends when value is moved or dropped. (So, not necessarily tied to scope.)
- Two references (e.g &x and &y) given a similar lifetime ('a) essentially denote that memory for both is still valid for a given
  area of our code (denoted by 'a). 
  
  

In [9]:
fn moved_b(){
    let a;
    {
        let b = String::from("Hello");
        // Because &T is Copy, assignment of b to a doesn't move
        // contents of b as the following assignment would:
        //
        // a = b
        //
        // Instead, we get, as the name implies, a reference to whatever b
        // is pointing to.
        a = &b
    } // b's memory is dropped here (end of scope). 
    // So, reference to it would be left dangling.
    // println!("{}", a);
}

moved_b()

#### E0106

An example of missing lifetime specifier:

In [13]:
/// No references passed in so ref returned would have to be returned from in function.
/// Also knows that references from inside the function would (probably? definitely?) that all
/// variables created inside the function will be cleaned when the function is finished (end of scope)
/// Applying 'static to it does not help (rust understands that we still return ref to local.)
fn missing_1() -> &i32 {
    let a = 1;
    &a
}

#### E0621

Mismatch in lifetime specifiers.

In [10]:
/// Conditionally return ref.
/// One possible solution: subtyping for lifetimes, i.e place a bound on
/// 'b to tell the compiler to *only* accept lifetimes >= 'b. 
fn condition_ref_return<'a, 'b>(a: &'a i32, b: &'b i32) -> &'a i32 {
    if a > b {
        a
    } else {
        b
    }
}

Error: lifetime mismatch

### Related Errors:

 - [E0106](https://doc.rust-lang.org/error-index.html#E0106)
 - [E0495](https://doc.rust-lang.org/error-index.html#E0495)
 - [E0515](https://doc.rust-lang.org/error-index.html#E0515)
 - [E0621](https://doc.rust-lang.org/error-index.html#E0621)

### Related Videos, Blogs

Videos:

- [YouTube tutorial 1](https://www.youtube.com/watch?v=rAl-9HwD858)
- [YouTube tutorial 2](https://www.youtube.com/watch?v=MSi3E5Z8oRw&list=WL&index=17)

Blogs, Github, others:

- [Pretzelhammer - Common Rust lifetime misconceptions](https://github.com/pretzelhammer/rust-blog/blob/master/posts/common-rust-lifetime-misconceptions.md)

### More examples:

1. Iterate over a collection of `T` and give out mutable references to items. Basically an `iter_mut`.

(Also an example of E0495)
Aside: There's an interesting question here. Why does the `&mut` case complain when the `&` case compiles just fine? Think of this and google it.

In [20]:
// Since struct is holding a ref, we *need* to specify how long that will live.
// Read as: IterWrapper lives *as long as* reference it contains lives. This *does* mean that
// `slice` can live longer, though. IterWrapper *can't*.
struct IterWrapper<'a, T> {
    slice: &'a [T],
}

// Associated item "Item" must be tied to our IterWrapper. In essense, we still need
// to explicitly state that item returned from next (which is a reference into our slice) 
// lives as long as the `slice` we we've wrapped.
impl<'a, T> Iterator for IterWrapper<'a, T>{
    type Item = &'a T;
    
    fn next(&mut self) -> Option<Self::Item> {
        if self.slice.is_empty() {
            return None;
        }
        // Grab element, trim slice.
        let element = self.slice.get(0);
        self.slice = &self.slice[1..];
        element
    }
}

fn test_iterwrapper(){
    let v = vec![1, 2, 3, 4];
    let iw = IterWrapper { slice: &v[..] };
    
    for value in iw {
         println!("{}", value);
    }
}

test_iterwrapper()

The struct for which we will implement `Iterator`:

In [2]:
struct MutIterWrapper<'a, T> {
    slice: &'a mut [T],
}

First stab at implementing `Iterator` for MutIterWrapper:

In [23]:
impl<'a, T> Iterator for MutIterWrapper<'a, T>{
    type Item = &'a mut T;
    
    fn next(&mut self) -> Option<Self::Item> {
        let result = self.slice.get_mut(0);
        self.slice = &mut self.slice[1..];
        result
    }
}

In [5]:
use std::mem;

impl<'a, T> Iterator for MutIterWrapper<'a, T>{
    type Item = &'a mut T;
    
    fn next(&mut self) -> Option<Self::Item> {
        // Replace self.slice with something else (empty slice)
        // while we work on its copy.
        let slice = mem::replace(&mut self.slice, &mut []);
        // then we replace self.slice with slice from which
        // we took our result
        let (result, rest) = slice.split_first_mut()?;
        self.slice = rest;
        Some(result)
    }
}

In [6]:
fn test_mutiterwrapper(){
    let mut v = vec![1, 2, 3, 4];
    {
        let iw = MutIterWrapper { slice: &mut v[..] };
    
        for value in iw {
             *value = *value + 1;
        }
    }
    for value in v {
        println!("{}", value);
    }
}

test_mutiterwrapper()

2
3
4
5


()

### todo?

- static lifetimes
- lifetime ellision